In [5]:
# 获取股票日 K
from urllib.request import urlopen  # python自带爬虫库
import json  # python自带的json数据库
from random import randint  # python自带的随机数库
import pandas as pd
pd.set_option('expand_frame_repr', False)  # 当列太多时不换行
pd.set_option('display.max_rows', 5000)  # 最多显示数据的行数

def get_k_line(stock_code, k_type = 'day', num = 640):
    randomStr = str(randint(10**15, (10**16)-1))
    url = f'http://web.ifzq.gtimg.cn/appstock/app/fqkline/get?_var=kline_{k_type}qfq&param={stock_code},{k_type},,,{num},qfq&r=0.{randomStr}'
    content = urlopen(url).read().decode()

    # ===将数据转换成dict格式
    content = content.split('=', maxsplit=1)[-1]
    content = json.loads(content)  # 自己去仔细看下这里面有什么数据
    
    # ===将数据转换成DataFrame格式
    k_data = content['data'][stock_code]
    if k_type in k_data:
        k_data = k_data[k_type]
    elif 'qfq' + k_type in k_data:  # qfq是前复权的缩写
        k_data = k_data['qfq' + k_type]
    else:
        raise ValueError('已知的key在dict中均不存在，请检查数据')
    df = pd.DataFrame(k_data)

    # ===对数据进行整理
    rename_dict = {0: 'candle_end_time', 1: 'open', 2: 'close', 3: 'high', 4: 'low', 5: 'amount', 6: 'info'}
    # 其中amount单位是手，说明数据不够精确
    df.rename(columns=rename_dict, inplace=True)
    df['prev_close'] = df['close'].shift(1)
    df['candle_end_time'] = pd.to_datetime(df['candle_end_time'])
    if 'info' not in df:
        df['info'] = None
    df = df[['candle_end_time', 'open', 'high', 'low', 'close', 'prev_close', 'amount', 'info']]
    return df

stock_code = 'sh000002'
df = get_k_line(stock_code)
df.head()
df.to_csv(f'{stock_code}_day_k.csv', index=False)

In [7]:
# 获取上证指数日 K 数据，包括前收盘价
df_sh000001 = get_k_line('sh000001', 'day', 2000)
df_sh000001.head()
df_sh000001.to_csv(f'sh000001_day_k.csv', index=False)


In [ ]:
# 将日线数据转化为月线数据，计算涨跌幅和交易天数

